In [1]:
import pandas as pd

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'
coursera_skills = 'coursera_skills/Coursera Skills To DB.xlsx'

In [2]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])].dropna()
        print(df_error['Parent'].unique().tolist())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [3]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Art
2,Business
3,Computer Science
4,Data Science


In [4]:
df_s1.describe(include='object')

,skill
count,1798
unique,1798
top,Rackspace
freq,1


In [5]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Data Analysis
1,Machine Learning
2,Algorithm
3,Artificial Intelligence
4,Big Data


In [6]:
df_s2.describe(include='object')

,skill
count,984
unique,984
top,TenFourFox
freq,1


In [7]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills = df_skills.dropna()
df_skills.describe()

,skill
count,2605
unique,2605
top,Security Assertion Markup Language (SAML)
freq,1


In [8]:
df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0] + 1
df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.head()

,skill_id,skill
0,126,Applied Science
1,139,Art
2,279,Business
3,459,Computer Science
4,589,Data Science


In [9]:
df_skills.describe(include='all')

,skill_id,skill
count,2605.000000,2605
unique,NaN,2605
top,NaN,Security Assertion Markup Language (SAML)
freq,NaN,1
mean,1303.000000,NaN
std,752.143049,NaN
min,1.000000,NaN
25%,652.000000,NaN
50%,1303.000000,NaN
75%,1954.000000,NaN


### Coursera Skills

In [10]:
df_c = pd.read_excel(coursera_skills)
df_c.head()

,skill_id,c_skill,Skill,Parent
0,457,Low Poly,Low Poly,3D Computer Graphics
1,462,Polygon Mesh,Polygon Mesh,3D Computer Graphics
2,466,Uv Mapping,Uv Mapping,3D Computer Graphics
3,467,Framing,Framing,3D Computer Graphics
4,469,3D Graphics Software,3D Graphics Software,3D Computer Graphics


In [11]:
df_coursera = df_c[['skill_id', 'c_skill']]
df_coursera = df_coursera.rename(columns={'c_skill': 'skill'})
df_coursera.head()

,skill_id,skill
0,457,Low Poly
1,462,Polygon Mesh
2,466,Uv Mapping
3,467,Framing
4,469,3D Graphics Software


In [12]:
df_others = df_c[['Skill']].rename(columns={'Skill': 'skill'})
df_others = df_others.drop_duplicates()
df_others = df_others.dropna()
df_others['skill_id'] = df_others.index + df_skills['skill_id'].max()
df_others.head()

,skill,skill_id
0,Low Poly,2605
1,Polygon Mesh,2606
2,Uv Mapping,2607
3,Framing,2608
4,3D Graphics Software,2609


In [13]:
# Combine
df_all = df_skills.append(df_others)
df_all = df_all.drop_duplicates()
df_all = df_all.reset_index(drop=True)
df_all = df_all[['skill_id', 'skill']]
df_all.head()

,skill_id,skill
0,126,Applied Science
1,139,Art
2,279,Business
3,459,Computer Science
4,589,Data Science


In [14]:
df_all.describe(include='all')

,skill_id,skill
count,9873.000000,9873
unique,NaN,9134
top,NaN,Voice User Interface
freq,NaN,2
mean,4953.429049,NaN
std,2862.082636,NaN
min,1.000000,NaN
25%,2469.000000,NaN
50%,4956.000000,NaN
75%,7434.000000,NaN


### Data Checking/Cleaning

In [15]:
parent_check(all_skills, df_skills)

[]
[]
[]
[]
[]
[]
[]


In [16]:
parent_check(digital_skills, df_skills)

[]
[]
[]
[]
[]
[]


In [17]:
parent_check(coursera_skills, df_all)

[]


### Extract Skill Relation

In [18]:
# Skills
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel = df_rel.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel.head()

,skill_1_id,skill_2_id
0,1306,1564
1,1799,1564
2,2373,459
3,2373,1413
4,55,126


In [19]:
df_rel.describe(include='all')

,skill_1_id,skill_2_id
count,3096.0,3096.000000
unique,2596.0,NaN
top,2057.0,NaN
freq,7.0,NaN
mean,NaN,1237.786822
std,NaN,744.511303
min,NaN,1.000000
25%,NaN,583.000000
50%,NaN,1295.500000
75%,NaN,1880.000000


In [22]:
# Include coursera skills
df_r3 = extract_relation(coursera_skills, df_all)
# Combine
df_rel_all = df_rel.append(df_r3[['Skill', 'Parent']])
df_rel_all = df_rel_all.drop_duplicates()
df_rel_all = df_rel_all.reset_index(drop=True)
df_rel_all = df_rel_all.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel_all.head()

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
0,1306,1564.0,NaN,NaN
1,1799,1564.0,NaN,NaN
2,2373,459.0,NaN,NaN
3,2373,1413.0,NaN,NaN
4,55,126.0,NaN,NaN


In [23]:
df_rel_all.describe(include='all')

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
count,3096.0,3096.000000,2349.0,2349.000000
unique,2596.0,NaN,2343.0,NaN
top,2057.0,NaN,3064.0,NaN
freq,7.0,NaN,2.0,NaN
mean,NaN,1237.786822,NaN,4159.479353
std,NaN,744.511303,NaN,2132.058722
min,NaN,1.000000,NaN,31.000000
25%,NaN,583.000000,NaN,2836.000000
50%,NaN,1295.500000,NaN,4186.000000
75%,NaN,1880.000000,NaN,5260.000000


### Extract Coursera Relation

In [25]:
df_crel = df_c[['skill_id', 'Skill']].rename(columns={'skill_id': 'coursera_skill_id', 'Skill': 'skill'})\
    .merge(df_all, how='left').drop(columns=['skill'])
df_crel.head()

,coursera_skill_id,skill_id
0,457,2605
1,462,2606
2,466,2607
3,467,2608
4,469,2609


In [26]:
df_crel.describe(include='all')

,coursera_skill_id,skill_id
count,8052.000000,8052.000000
mean,3564.818058,5781.290487
std,2097.017109,2493.571410
min,0.000000,2.000000
25%,1756.750000,3855.750000
50%,3495.500000,5874.500000
75%,5352.250000,7889.250000
max,7301.000000,9905.000000


### Export

In [27]:
df_skills.to_csv('skills_db/skill.csv', index=False)
df_rel.to_csv('skills_db/skill_tree.csv', index=False)
df_all.to_csv('skills_db/all_skill.csv', index=False)
df_rel_all.to_csv('skills_db/all_skill_tree.csv', index=False)
df_coursera.to_csv('skills_db/coursera_skill.csv', index=False)
df_crel.to_csv('skills_db/coursera_map.csv', index=False)